# Data loading 
Next cells are used to load the data we need.

In [1]:
import os
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import sys
import matplotlib.pyplot as plt
sys.path.append('../../../')


In [2]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('../../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [3]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [4]:
ICM_df = pd.read_csv('../../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [5]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [6]:
grouped_users = dataset.groupby(['row']).count()
sorted_users = grouped_users.sort_values(by=['col'], ascending=True)
sorted_users = sorted_users.index.to_numpy()
block_size = int(len(sorted_users)*0.75)

start_pos_group_0 = 0*block_size
end_pos_group_0 = min((0+1)*block_size, len(sorted_users))
users_in_group_0 = sorted_users[start_pos_group_0:end_pos_group_0]
users_not_in_group_0_flag = np.isin(sorted_users, users_in_group_0, invert = True)
users_not_in_group_0 = sorted_users[users_not_in_group_0_flag]


start_pos_group_1 = 1*block_size
end_pos_group_1 = min((1+1)*block_size, len(sorted_users))
users_in_group_1 = sorted_users[start_pos_group_1:end_pos_group_1]
users_not_in_group_1_flag = np.isin(sorted_users, users_in_group_1, invert = True)
users_not_in_group_1 = sorted_users[users_not_in_group_1_flag]

In [7]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train_1, URM_validation_1 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 1)

URM_train_2, URM_validation_2 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 2)

URM_train_3, URM_validation_3 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 3)

URM_train_4, URM_validation_4 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 4)

URM_train_5, URM_validation_5 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 5)

URM_train_vector = [URM_train_1,URM_train_2,URM_train_3,URM_train_4,URM_train_5]

evaluator_validation_1 = EvaluatorHoldout(URM_validation_1, cutoff_list=[10], ignore_users = users_not_in_group_1)
evaluator_validation_2 = EvaluatorHoldout(URM_validation_2, cutoff_list=[10], ignore_users = users_not_in_group_1)
evaluator_validation_3 = EvaluatorHoldout(URM_validation_3, cutoff_list=[10], ignore_users = users_not_in_group_1)
evaluator_validation_4 = EvaluatorHoldout(URM_validation_4, cutoff_list=[10], ignore_users = users_not_in_group_1)
evaluator_validation_5 = EvaluatorHoldout(URM_validation_5, cutoff_list=[10], ignore_users = users_not_in_group_1)

Evaluator_vector = [evaluator_validation_1, evaluator_validation_2, evaluator_validation_3, evaluator_validation_4, evaluator_validation_5]


EvaluatorHoldout: Ignoring 5960 Users
EvaluatorHoldout: Ignoring 5960 Users
EvaluatorHoldout: Ignoring 5960 Users
EvaluatorHoldout: Ignoring 5960 Users
EvaluatorHoldout: Ignoring 5960 Users


In [9]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
i=1
for URM_train, evaluator in zip(URM_train_vector,Evaluator_vector):
    
    URM_train_ICM_all_RP3 = sps.vstack([URM_train, ICM_all.T])
    recommender = RP3betaRecommender(URM_train_ICM_all_RP3)
    recommender.fit(topK = 396, alpha = 0.41137245632718406, beta = 0.29679690096781264, implicit = False)

    #recommender.URM_train = URM_train.tocsr()
    
    result_dict, _ = evaluator.evaluateRecommender(recommender)
    print(result_dict)
    
    filename = 'RP3_fold_number' + str(i)
    recommender.save_model('RP3_saved_group_1/', file_name = filename)
    i = i+1

RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
EvaluatorHoldout: Processed 1951 ( 100.00% ) in 1.67 sec. Users per second: 1166
{10: {'ROC_AUC': 0.3111641974404665, 'PRECISION': 0.08200922603792823, 'PRECISION_RECALL_MIN_DEN': 0.1506712064631076, 'RECALL': 0.13602680592253433, 'MAP': 0.07112665410906843, 'MRR': 0.24411982459137732, 'NDCG': 0.12154824925020437, 'F1': 0.10232669319666743, 'HIT_RATE': 0.8200922603792927, 'ARHR': 0.3048654332739419, 'NOVELTY': 0.005352389270362488, 'AVERAGE_POPULARITY': 0.1325255443139527, 'DIVERSITY_MEAN_INTER_LIST': 0.9892108451944434, 'DIVERSITY_HERFINDAHL': 0.9988703817595677, 'COVERAGE_ITEM': 0.26398460057747836, 'COVERAGE_ITEM_CORRECT': 0.04192492781520693, 'COVERAGE_USER': 0.9818822345244087, 'COVERAGE_USER_CORRECT': 0.5027679919476598, 'DIVERSITY_GINI': 0.1182689039774443, 'SHANNON_ENTROPY': 11.49042985068578}}
RP3betaRecommender: Saving model in file 'RP3_saved_group_1/RP3_fold_numb